In [1]:
import optuna
import xgboost as xgb
import sklearn
import datetime
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer


def objective(trial):
    
    
    param = {
        "objective": 'binary:logistic',
        'reg_lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'reg_alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma':trial.suggest_loguniform('gamma', 1e-3,1),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [i/10.0 for i in range(4,11)]),
        'subsample': trial.suggest_categorical('subsample', [i/10.0 for i in range(4,11)]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.300000012]),
        'n_estimators': trial.suggest_int('n_estimators',100,500),
        'max_depth': trial.suggest_categorical('max_depth', [5,6,7,9,11,13,15,17,20]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param)  
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

In [2]:
from sklearn.datasets import make_classification
X,y = make_classification(n_samples=500, n_features=15, n_informative=15, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=2,class_sep=0.75,flip_y=0,weights=[0.5,0.5], random_state=17)

In [14]:
# https://saturncloud.io/docs/examples/machinelearning/xgboost-training/

In [27]:
import dask.array as da
import xgboost as xgb

from dask.distributed import Client, wait
from dask_ml.metrics import mean_absolute_error

cluster = dask.distributed.LocalCluster(n_workers=2, threads_per_worker=1)
client = dask.distributed.Client(cluster)
cluster

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\distributed\node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61167 instead
  http_address["port"], self.http_server.port


In [48]:
import pandas as pd
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [56]:
import dask.dataframe as dd
# df = dd.read_parquet('s3://...')

# # Split into training and testing data
# train, test = df.random_split([0.8, 0.2])

# # Separate labels from data
# train_labels = train.x > 0
# test_labels = test.x > 0

# del train['x']  # remove informative column from data
# del test['x']  # remove informative column from data

from xgboost import XGBRegressor  # change import
from dask_ml.xgboost import XGBRegressor

est = XGBRegressor()
est.fit(X_train, y_train)

prediction = est.predict(X_test)

AttributeError: 'DataFrame' object has no attribute 'to_delayed'

In [58]:
import numpy as np
import dask_ml.model_selection as dcv
import dask_xgboost
import sklearn.model_selection
from distributed import Client

client = Client(processes=False)

x = np.random.randn(100, 2)
y = np.random.randint(0, 1, 100)

params = {'max_depth': [2, 3]}

clf = dcv.GridSearchCV(dask_xgboost.XGBClassifier(), params, cv=sklearn.model_selection.KFold(n_splits=2),
                       scoring='neg_mean_squared_error', scheduler=client)
clf.fit(x, y)

C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\distributed\node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62391 instead
  http_address["port"], self.http_server.port
distributed.worker - WARNING - Compute Failed
Function:  fit_and_score
args:      (XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None), <dask_ml.model_selection.m

distributed.utils - ERROR - "('xgbclassifier-fit-score-db69da5b1e6bba4675cc50adc60d08f2', 0, 0)"
Traceback (most recent call last):
  File "C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\distributed\utils.py", line 668, in log_errors
    yield
  File "C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\distributed\scheduler.py", line 3986, in add_worker
    typename=types[key],
KeyError: "('xgbclassifier-fit-score-db69da5b1e6bba4675cc50adc60d08f2', 0, 0)"
distributed.core - ERROR - Exception while handling op register-worker
Traceback (most recent call last):
  File "C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\distributed\core.py", line 501, in handle_comm
    result = await result
  File "C:\Users\004567\Anaconda3\envs\root3\lib\site-packages\distributed\scheduler.py", line 3986, in add_worker
    typename=types[key],
KeyError: "('xgbclassifier-fit-score-db69da5b1e6bba4675cc50adc60d08f2', 0, 0)"


KeyboardInterrupt: 

In [73]:
# evaluate the effect of the number of threads
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import time
from matplotlib import pyplot

start = time.time()
model = XGBClassifier(nthread=-1)
model.fit(X_train, y_train)
elapsed = time.time() - start
print(elapsed)


[23:53:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.1422595977783203


In [75]:
start = time.time()
model = XGBClassifier()
model.fit(X_train, y_train)
elapsed = time.time() - start
print(elapsed)

[23:53:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.2745633125305176


In [ ]:
# https://machinelearningmastery.com/best-tune-multithreading-support-xgboost-python/

In [31]:
# load dataset
import pandas as pd
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# model = xgb.XGBClassifier()  
# model.fit(X_train, y_train)
# preds = model.predict(X_test)
# accuracy = accuracy_score(y_test, preds)
# accuracy

In [ ]:
random_study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)

In [ ]:
random_study.optimize(objective, n_trials=100)

In [ ]:
random_study.trials_dataframe()

In [ ]:
tstamp1 = random_study.trials_dataframe()['datetime_complete'].min()
tstamp2 = random_study.trials_dataframe()['datetime_complete'].max()

tstamp1 - tstamp2 if tstamp1 > tstamp2 else tstamp2 - tstamp1

In [ ]:
random_study.best_value

In [ ]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100)

In [ ]:
study.best_value

In [ ]:
tstamp1 = study.trials_dataframe()['datetime_complete'].min()
tstamp2 = study.trials_dataframe()['datetime_complete'].max()

tstamp1 - tstamp2 if tstamp1 > tstamp2 else tstamp2 - tstamp1